# Piperack Generator 1.0

This notebook generates a piperack structure based on user-defined inputs and applies relevant loads and design parameters for analysis in STAAD.

## Workflow

1. **Configure Inputs**  
   Edit the following input sections in this notebook:  
   - **Basic Structure**: Define the overall geometry and layout.  
   - **Tiers**: Specify tier configurations and elevations.  
   - **Flares, Ducts, and Walkways**: Include additional components as needed.  
   - **Foundation**: Set foundation parameters.  
   - **Profiles**: Use STAAD-compatible profile names for members.  
   - **Supports and Specifications**: Define support conditions and other specifications.  

2. **Generate Piperack**  
   Run the generator to create the piperack model based on the provided inputs.

3. **Correct Self-Weight**  
   Correct self-weight to members (steel and concrete) using the Member Selector.

4. **Apply Loads**  
   - Assign seismic loads as per design requirements.

5. **Set Design Parameters** (via STAAD Helper and Parameter Selector)  
   - **Parameters 1 and 3**: Apply to main members.  
   - **Parameters 2 and 4**: Apply to local members.

6. **Optimize Design**  
   Use the Design Optimiser to assign appropriate sections to the structure.

---

**Note**: Ensure all inputs align with STAAD conventions for accurate analysis and design.

## Module imports and connection to STAAD

In [1]:
from collections import defaultdict
import pyperclip
from IPython.display import display, Markdown
from base.helper.general import *
from base.geometry_base.line import *
from base.geometry_base.rectangle import *
from base.staad_base.geometry import *
from base.structural_elements.beam import *
from base.structural_elements.column import *
from base.structural_elements.brace import *
from base.piperack.portal import *
from base.piperack.piperack import *
from base.piperack.tier import *
from base.load.nodal_load import *
from base.load.uniform_load import *
from base.staad_base.design import *
from base.staad_base.property import *
from output.md_output import *
from base.staad_base.optimise_member import *
from base.insturmentation_elements.duct import *
from base.structural_elements.walkway import *
from base.piping_elements.flare import *
from base.structural_elements.tree_support import *

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output
support = STAAD_objects.support
load = STAAD_objects.load

add_beams = add_beams_fn(geometry=geometry)
select_beams = select_beams_fn(geometry=geometry)
assign_profile = assign_profile(property=property)
assign_specification = assign_specification(property=property)

## Piperack Inputs

### Portal and Structure

In [2]:
base_point_of_first_portal : Point3D = Point3D(0,0,0)
width_of_piperack = 8
portal_distances = [0,8,16,24,32,40,48]
column_distances = [0,width_of_piperack]
braces_placement = [False,True,False,False,True,False]
brace_pattern = BracePattern.X_Pattern
bracket_size = 2
max_expansion_bay_length = 8

In [3]:
uniform_operating_load = UniformLoad(load_case=LoadCase.OperatingLoad)
uniform_empty_load = UniformLoad(load_case=LoadCase.EmptyLoad)

uniform_tg_gx = UniformLoad(load_case=LoadCase.ThermalGravity_GX,direction=MemberDirection.GX)
uniform_tg_gz = UniformLoad(load_case=LoadCase.ThermalGravity_GZ,direction=MemberDirection.GZ)
uniform_tl_gx = UniformLoad(load_case=LoadCase.ThermalLateral_GX,direction=MemberDirection.GX)
uniform_tl_gz = UniformLoad(load_case=LoadCase.ThermalLateral_GZ,direction=MemberDirection.GZ)

uniform_clt_gy = UniformLoad(load_case=LoadCase.ContigencyLoadTransverse,direction=MemberDirection.GY)
uniform_clt_gz = UniformLoad(load_case=LoadCase.ContigencyLoadTransverse,direction=MemberDirection.GZ)

conc_operating_load = ConcentratedLoad(load_case=LoadCase.OperatingLoad)
conc_empty_load = ConcentratedLoad(load_case=LoadCase.EmptyLoad)

conc_tg_gx = ConcentratedLoad(load_case=LoadCase.ThermalGravity_GX,direction=MemberDirection.GX)
conc_tg_gz = ConcentratedLoad(load_case=LoadCase.ThermalGravity_GZ,direction=MemberDirection.GZ)
conc_tl_gx = ConcentratedLoad(load_case=LoadCase.ThermalLateral_GX,direction=MemberDirection.GX)
conc_tl_gz = ConcentratedLoad(load_case=LoadCase.ThermalLateral_GZ,direction=MemberDirection.GZ)

# Tier wind loads to be implemented
wl_gx = NodalLoad(load_case=LoadCase.WindTier_GX)
wl_gx_o = NodalLoad(load_case=LoadCase.WindTier_GX_Opposite)

#### Tiers

##### Piping Tiers

In [4]:

tiers = [
    Tier(base=base_point_of_first_portal.shift_y(3),
         loads=[uniform_operating_load.set_force_value(-0.4)],
         wind_loads = [wl_gx.set_fx(1.42),wl_gx_o.set_fx(-1.42)],
         ).set_intermediate_transverse_beam(False),

    Tier(base=base_point_of_first_portal.shift_y(6),
         loads=[uniform_operating_load.set_force_value(-0.4)],
         wind_loads = [wl_gx.set_fx(1.316),wl_gx_o.set_fx(-1.316)],
         ).set_intermediate_transverse_beam(False),

    Tier(base=base_point_of_first_portal.shift_y(9.5),
         loads=[uniform_operating_load.set_force_value(-0.35)],
         wind_loads = [wl_gx.set_fx(1.014),wl_gx_o.set_fx(-1.014)]
         ).set_bracket_provision(True),

    Tier(base=base_point_of_first_portal.shift_y(12),
         loads=[uniform_operating_load.set_force_value(-0.3)],
         wind_loads = [wl_gx.set_fx(1.118),wl_gx_o.set_fx(-1.118)]
         ).set_bracket_provision(True),

    Tier(base=base_point_of_first_portal.shift_y(14.5),
         tier_type= TierType.ElectricalIntrumentation,
         wind_loads = [wl_gx.set_fx(1.635),wl_gx_o.set_fx(-1.635)]
         ).set_intermediate_transverse_beam(False),

    Tier(base=base_point_of_first_portal.shift_y(17.5),
         tier_type= TierType.Flare,
         wind_loads = [wl_gx.set_fx(2.549),wl_gx_o.set_fx(-2.549)]
         ).set_intermediate_transverse_beam(False),
]

tiers = sorted(tiers,key = lambda tier_x : tier_x.base.y)

for tier_x in tiers:
    if(tier_x.loads):
        tier_op_load_x = tier_x.loads[0]
        if(tier_op_load_x):
            tier_x.add_load(uniform_empty_load.set_force_value(tier_op_load_x.force_value*0.4))
            tier_x.add_load(uniform_tg_gx.set_force_value(tier_op_load_x.force_value*(-0.025)))
            tier_x.add_load(uniform_tg_gz.set_force_value(tier_op_load_x.force_value*(-0.125)))
            tier_x.add_load(uniform_tl_gx.set_force_value(tier_op_load_x.force_value*(-0.05)))
            tier_x.add_load(uniform_tl_gz.set_force_value(tier_op_load_x.force_value*(-0.05)))

for tier_x in tiers:
    if(tier_x.loads):
        tier_op_load_x = tier_x.loads[0]
        if(tier_op_load_x):
            tier_x.add_clt_load(uniform_clt_gy.set_force_value(tier_op_load_x.force_value*0.4))
            tier_x.add_clt_load(uniform_clt_gz.set_force_value(tier_op_load_x.force_value*(-1*0.04)))
            
tier_elevations = [tier.base.y for tier in tiers]
tier_dict = { tier.base.y : tier for tier in tiers }

display(Markdown(create_tiers_markdown_table(tiers)))

# Tier List

| Tier Number | Elevation (y) | Load Type |
|-------------|---------------|-----------|
| Tier 1 | 3 | UniformLoad(direction=GY, force=-0.4, d1=0, d2=0, d3=0, load_case=OperatingLoad), UniformLoad(direction=GY, force=-0.16000000000000003, d1=0, d2=0, d3=0, load_case=EmptyLoad), UniformLoad(direction=GX, force=0.010000000000000002, d1=0, d2=0, d3=0, load_case=ThermalGravity_GX), UniformLoad(direction=GZ, force=0.05, d1=0, d2=0, d3=0, load_case=ThermalGravity_GZ), UniformLoad(direction=GX, force=0.020000000000000004, d1=0, d2=0, d3=0, load_case=ThermalLateral_GX), UniformLoad(direction=GZ, force=0.020000000000000004, d1=0, d2=0, d3=0, load_case=ThermalLateral_GZ) |
| Tier 2 | 6 | UniformLoad(direction=GY, force=-0.4, d1=0, d2=0, d3=0, load_case=OperatingLoad), UniformLoad(direction=GY, force=-0.16000000000000003, d1=0, d2=0, d3=0, load_case=EmptyLoad), UniformLoad(direction=GX, force=0.010000000000000002, d1=0, d2=0, d3=0, load_case=ThermalGravity_GX), UniformLoad(direction=GZ, force=0.05, d1=0, d2=0, d3=0, load_case=ThermalGravity_GZ), UniformLoad(direction=GX, force=0.020000000000000004, d1=0, d2=0, d3=0, load_case=ThermalLateral_GX), UniformLoad(direction=GZ, force=0.020000000000000004, d1=0, d2=0, d3=0, load_case=ThermalLateral_GZ) |
| Tier 3 | 9.5 | UniformLoad(direction=GY, force=-0.35, d1=0, d2=0, d3=0, load_case=OperatingLoad), UniformLoad(direction=GY, force=-0.13999999999999999, d1=0, d2=0, d3=0, load_case=EmptyLoad), UniformLoad(direction=GX, force=0.008749999999999999, d1=0, d2=0, d3=0, load_case=ThermalGravity_GX), UniformLoad(direction=GZ, force=0.04375, d1=0, d2=0, d3=0, load_case=ThermalGravity_GZ), UniformLoad(direction=GX, force=0.017499999999999998, d1=0, d2=0, d3=0, load_case=ThermalLateral_GX), UniformLoad(direction=GZ, force=0.017499999999999998, d1=0, d2=0, d3=0, load_case=ThermalLateral_GZ) |
| Tier 4 | 12 | UniformLoad(direction=GY, force=-0.3, d1=0, d2=0, d3=0, load_case=OperatingLoad), UniformLoad(direction=GY, force=-0.12, d1=0, d2=0, d3=0, load_case=EmptyLoad), UniformLoad(direction=GX, force=0.0075, d1=0, d2=0, d3=0, load_case=ThermalGravity_GX), UniformLoad(direction=GZ, force=0.0375, d1=0, d2=0, d3=0, load_case=ThermalGravity_GZ), UniformLoad(direction=GX, force=0.015, d1=0, d2=0, d3=0, load_case=ThermalLateral_GX), UniformLoad(direction=GZ, force=0.015, d1=0, d2=0, d3=0, load_case=ThermalLateral_GZ) |
| Tier 5 | 14.5 | None |
| Tier 6 | 17.5 | None |


##### E&I And Flare Details

In [5]:
flares = [
    Flare(lines=[Line3D(Point3D(2.5,17.5,portal_distances[0]),Point3D(2.5,17.5,portal_distances[-1]))],support_member=True,design_load=3),
    Flare(lines=[Line3D(Point3D(5,17.5,portal_distances[0]),Point3D(5,17.5,portal_distances[-1]))],support_member=False,design_load=0.35),
    Flare(lines=[Line3D(Point3D(6,17.5,portal_distances[0]),Point3D(6,17.5,portal_distances[-1]))],support_member=False,design_load=0.3),
]

walkways = [
    SteelWalkway(edge_line=Line3D(Point3D(1,14.5,portal_distances[0]),Point3D(1,14.5,portal_distances[-1]))),
    SteelWalkway(edge_line=Line3D(Point3D(4.5,14.5,portal_distances[0]),Point3D(4.5,14.5,portal_distances[-1]))),
]

ducts = [
    InstrumentationDuct(width=1.2,height=0.4,edge_line=Line3D(Point3D(6,14.5,portal_distances[0]),Point3D(6,14.5,portal_distances[-1])))
]

electric_trees = [
    TreeSupportMember(line=Line3D(Point3D(3,14.5,portal_distances[0]),Point3D(3,14.5,portal_distances[-1])),)
]

ww_support_members = [line for ww_x in walkways for line in ww_x.get_member_lines()]
flare_support_members = [line for flare in flares if flare.support_member for line in flare.lines]
duct_support_members = [line for duct_x in ducts for line in duct_x.get_member_lines()]
tree_support_members = [et.line for et in electric_trees if et.support_member]

#### Longitudinal Beam Elevations


In [6]:
long_beam_elevations = [*[tiers[tier_i].base.mid(tiers[tier_i+1].base).y for tier_i in range(len(tiers)-1)],tier_elevations[-1]]

display(long_beam_elevations)

[4.5, 7.75, 10.75, 13.25, 16.0, 17.5]

### Foundation

In [7]:
pedestal_height = 2
foundation_depth = 1

## Structural Inputs
### Materials

In [8]:
assign_material_steel = assign_material(property)('STEEL')
assign_material_concrete = assign_material(property)('CONCRETE')

### Profiles

In [9]:
concrete_profile_list = {
    PiperackMembers.Pedestals : Rectangle(1.2,0.8)
}

steel_profile_list = {
    PiperackMembers.Columns : 'WPB700X300X240.51',
    PiperackMembers.TierBeams : 'WPB600X300X128.79',
    PiperackMembers.LongitudinalBeams : 'WPB250X250X73.14',
    PiperackMembers.PlanBracing : '100X100X6.0SHS',
    PiperackMembers.VerticalBracing : '150X150X8.0SHS',
    PiperackMembers.Stubs : 'ISMB300',
    PiperackMembers.IntermediateTransverseBeams : 'ISMB300',
    PiperackMembers.IntermediateLongitudinalBeams : 'ISMB300',
    PiperackMembers.FlareSupportMembers : 'WPB700X300X240.51',
    PiperackMembers.TreeSupportMembers : 'ISMC400',
    PiperackMembers.DuctSupportMembers : 'ISMC250',
    PiperackMembers.WWSupportMembers : 'ISMC200',
    PiperackMembers.BracketBeams : 'ISMB400',
    PiperackMembers.BracketBraces : '100X100X6.0SHS',
}

profile_ids = {}

for member_type,profile in concrete_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        profile_ids[member_type] = create_concrete_beam_property(property,profile)

for member_type,profile in steel_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        if('SHS' in profile):
            profile_ids[member_type] = create_steel_beam_property(property,35,profile,0,0,0)
        else:
            profile_ids[member_type] = create_steel_beam_property(property,10,profile,0,0,0)
        
display(Markdown(create_profile_markdown_table(concrete_profile_list, steel_profile_list, profile_ids)))

## Profile List

| Member Type | Profile | Profile ID |
|-------------|---------|------------|
| 2 | Rectangle : length = 1.2 and width = 0.8 | 1 |
| 1 | WPB700X300X240.51 | 2 |
| 0 | WPB600X300X128.79 | 3 |
| 3 | WPB250X250X73.14 | 4 |
| 5 | 100X100X6.0SHS | 6 |
| 4 | 150X150X8.0SHS | 7 |
| 6 | ISMB300 | 5 |
| 7 | ISMB300 | 5 |
| 8 | ISMB300 | 5 |
| 9 | WPB700X300X240.51 | 2 |
| 11 | ISMC400 | 12 |
| 10 | ISMC250 | 13 |
| 12 | ISMC200 | 14 |
| 13 | ISMB400 | 8 |
| 14 | 100X100X6.0SHS | 6 |


### Supports

In [10]:
support_id = support.CreateSupportFixed()

### Specifications

#### Member Releases

In [11]:
start_release_spec = property.CreateMemberReleaseSpec(0,set_DOFReleaseArray())
end_release_spec = property.CreateMemberReleaseSpec(1,set_DOFReleaseArray())
set_start_end_release = get_start_end_release_function(property=property,start_release_spec=start_release_spec,end_release_spec=end_release_spec)

#### Truss and Offset

In [12]:
foundation_depth = 1
truss_spec = property.CreateMemberTrussSpec()
member_offset_spec = property.CreateMemberOffsetSpec(0,0,0,foundation_depth,0)

display(Markdown(create_spec_markdown_table(start_release_spec,end_release_spec,truss_spec,member_offset_spec)))

## Member Specifications Created

| Specification Type | Details |
|-------------------|---------|
| Start Release | 1 |
| End Release | 2 |
| Truss | 3 |
| Offset Member | 0 |


## Member Creation

### Primary Members

In [13]:
#portals 
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []
plan_braces = []
vertical_braces = []
portal_beam_ids = [] 
portal_column_ids = [] 
portal_pedestal_ids = [] 
support_node_ids = []
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []
plan_brace_ids = []
vertical_brace_ids = []
portal_count = len(portal_distances)

# Create first portal
portal : PiperackPortal = PiperackPortal(base=base_point_of_first_portal) 

for z_column in column_distances:
    support_point = Point3D(z_column,0,0)-Point3D(0,pedestal_height,0)
    portal.add_pedestal(Column3D(base=support_point,height=pedestal_height))
    portal.add_column(Column3D(base=Point3D(z_column,0,0),height=tier_elevations[-1])) 
    
for tier_x in tier_elevations:
    for column_x in range(len(column_distances)-1):
        portal.add_beam(Beam3D(start=Point3D(column_distances[column_x],tier_x,0),end=Point3D(column_distances[column_x+1],tier_x,0))) 

# Create all portals by shifting the first portal
portals = [portal.shift(Point3D(0,0,dist)) for dist in portal_distances]

# Create longitudinal beams between portals
for portal_i in range(len(portals)-1):
    for z_column in column_distances:
        for long_beam in long_beam_elevations:
            long_beam_x = Beam3D(start=Point3D(z_column,long_beam,portals[portal_i].base.z),end=Point3D(z_column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)


In [14]:
# Add portal members to model and collect IDs
for portal in portals:
    portal_beam_ids = [*portal_beam_ids,*add_beams(portal.beams)]
    portal_column_ids = [*portal_column_ids,*add_beams(portal.columns)]
    portal_pedestal_ids = [*portal_pedestal_ids,*add_beams(portal.pedestals)]

# Assign profiles to portal members
portal_beam_assign = assign_profile(portal_beam_ids,profile_ids[PiperackMembers.TierBeams])
portal_column_assign = assign_profile(portal_column_ids,profile_ids[PiperackMembers.Columns])
portal_pedestal_assign = assign_profile(portal_pedestal_ids,profile_ids[PiperackMembers.Pedestals])

# Add longitudinal beams and assign properties
long_beam_ids = add_beams(longitudinal_beams)
long_release = set_start_end_release(long_beam_ids)
long_assign = assign_profile(long_beam_ids,profile_ids[PiperackMembers.LongitudinalBeams])

# Add support nodes
for portal in portals:
    for pedestal in portal.pedestals:
        support_node_ids.append(add_support_node(geometry,support,pedestal.start,support_id))

# Assign specifications
assign_specification(portal_pedestal_ids,member_offset_spec)

# Create comprehensive markdown table
markdown_output = "## Primary Members Created\n\n"
markdown_output += "| Member Types     | IDs                  | Profile Assignment     | Release Assignment     | Notes |\n"
markdown_output += "| ---              | ---                  | ---                    | ---                    | ---   |\n"
markdown_output += f"| Portal Columns   | {portal_column_ids}  | {portal_column_assign} | -                      | Vertical support members |\n"
markdown_output += f"| Portal Pedestals | {portal_pedestal_ids}| {portal_pedestal_assign}| -                     | Foundation connections |\n"
markdown_output += f"| Portal Beams     | {portal_beam_ids}    | {portal_beam_assign}   | -                      | Transverse tier beams |\n"
markdown_output += f"| Long Beams       | {long_beam_ids}      | {long_assign}          | {long_release}         | Longitudinal beams between portals |\n"
markdown_output += f"| Support Nodes    | {support_node_ids}   | -                      | -                      | Foundation support points |\n"

display(Markdown(markdown_output))

## Primary Members Created

| Member Types     | IDs                  | Profile Assignment     | Release Assignment     | Notes |
| ---              | ---                  | ---                    | ---                    | ---   |
| Portal Columns   | [7, 8, 17, 18, 27, 28, 37, 38, 47, 48, 57, 58, 67, 68]  | [True, True, True, True, True, True, True, True, True, True, True, True, True, True] | -                      | Vertical support members |
| Portal Pedestals | [9, 10, 19, 20, 29, 30, 39, 40, 49, 50, 59, 60, 69, 70]| [True, True, True, True, True, True, True, True, True, True, True, True, True, True]| -                     | Foundation connections |
| Portal Beams     | [1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 31, 32, 33, 34, 35, 36, 41, 42, 43, 44, 45, 46, 51, 52, 53, 54, 55, 56, 61, 62, 63, 64, 65, 66]    | [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]   | -                      | Transverse tier beams |
| Long Beams       | [73, 76, 79, 82, 85, 86, 89, 92, 95, 98, 101, 102, 104, 106, 108, 110, 112, 113, 115, 117, 119, 121, 123, 124, 126, 128, 130, 132, 134, 135, 137, 139, 141, 143, 145, 146, 148, 150, 152, 154, 156, 157, 159, 161, 163, 165, 167, 168, 170, 172, 174, 176, 178, 179, 181, 183, 185, 187, 189, 190, 192, 194, 196, 198, 200, 201, 203, 205, 207, 209, 211, 212]      | [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]          | [(True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True)]         | Longitudinal beams between portals |
| Support Nodes    | [15, 16, 31, 32, 47, 48, 63, 64, 79, 80, 95, 96, 111, 112]   | -                      | -                      | Foundation support points |


### Secondary Members

In [15]:
# Group longitudinal beams by Y coordinate
long_beams_y = group_beams_by_y(longitudinal_beams)

# Create vertical braces    
if(brace_pattern == BracePattern.V_Pattern):
    previous_y = None
    for y_i,tier_y in long_beams_y.items():
        base = portal.base.y if not previous_y else previous_y
        long_beams_sorted = beams_sorted_yxz(tier_y)

        for beam_i in range(len(long_beams_sorted)-1):
            portal_i = beam_i%(len(portals)-1)
            if(braces_placement[portal_i]):
                
                long_beam_i : Beam3D = long_beams_sorted[beam_i]

                vb_1 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.start.z),
                                end=long_beam_i.mid())
                vb_2 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.end.z),
                                end=long_beam_i.mid())
                    
                vertical_braces.append(vb_1)
                vertical_braces.append(vb_2)

        previous_y = y_i
        
elif(brace_pattern == BracePattern.X_Pattern):
# Initialize variables
    sorted_y = sorted(long_beams_y.keys())  # Sort keys of long_beams_y
    vertical_braces = []  # Ensure vertical_braces is defined (assuming it's a list)

    for portal_i, current_y_key in enumerate(sorted_y):
        tier_y = long_beams_y[current_y_key]
        
        # Determine previous and next y-keys
        previous_y_key = sorted_y[portal_i - 1] if portal_i > 0 else portal.base.y
        next_y_key = sorted_y[portal_i + 1] if portal_i + 1 < len(sorted_y) else None
        
        # Process only even-indexed groups
        if portal_i % 2 == 0:
            base_y = portal.base.y if previous_y_key is None else previous_y_key
            long_beams_sorted = beams_sorted_yxz(tier_y)
            
            # Iterate through beams (excluding the last one)
            for beam_idx in range(len(long_beams_sorted) - 1):
                portal_idx = beam_idx % (len(portals) - 1)
                
                if braces_placement[portal_idx]:
                    current_beam = long_beams_sorted[beam_idx]
                    
                    pt_1 = Point3D(current_beam.start.x, base_y, current_beam.start.z)
                    pt_2 = current_beam.mid()
                    angle_with_hzl_1 = Line3D(pt_1,pt_2).angle_with_xy_plane()
                    
                    # Create vertical braces from base
                    if(30<=angle_with_hzl_1<=60):
                        vb_1 = Beam3D(
                            start=Point3D(current_beam.start.x, base_y, current_beam.start.z),
                            end=current_beam.mid()
                        )
                        vb_2 = Beam3D(
                            start=Point3D(current_beam.start.x, base_y, current_beam.end.z),
                            end=current_beam.mid()
                        )
                        
                        vertical_braces.extend([vb_1, vb_2])
                    
                    else:
                        total_z = abs(pt_4.z-pt_3.z)
                        vb_5 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z+total_z/2),
                            end=current_beam.mid()
                        )
                        vb_6 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z-total_z/2),
                            end=current_beam.mid()
                        )
                        vb_7 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z).shift_y(current_beam.start.y),
                            end=vb_5.start
                        )
                        vb_8 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z).shift_y(current_beam.start.y),
                            end=vb_6.start
                        )
                        vertical_braces.extend([vb_5, vb_6 , vb_7 , vb_8])
                        
                    # Add braces to next y-key if it exists
                    if (next_y_key is not None):
                        pt_3 = Point3D(current_beam.start.x, next_y_key, current_beam.start.z)
                        pt_4 = current_beam.mid()
                        angle_with_hzl_2 = Line3D(pt_3,pt_4).angle_with_xy_plane()
                        
                        if(30<=angle_with_hzl_2<=60):
                        
                            vb_3 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z),
                                end=current_beam.mid()
                            )
                            vb_4 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z),
                                end=current_beam.mid()
                            )
                            
                            vertical_braces.extend([vb_3, vb_4])
                        else:
                            total_z = abs(pt_4.z-pt_3.z)
                            vb_5 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z+total_z/2),
                                end=current_beam.mid()
                            )
                            vb_6 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z-total_z/2),
                                end=current_beam.mid()
                            )
                            vb_7 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z).shift_y(current_beam.start.y),
                                end=vb_5.start
                            )
                            vb_8 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z).shift_y(current_beam.start.y),
                                end=vb_6.start
                            )
                            vertical_braces.extend([vb_5, vb_6 , vb_7 , vb_8])
                            

# Add vertical braces and assign properties
vertical_brace_ids = add_beams(vertical_braces)
vertical_brace_spec = assign_specification(vertical_brace_ids,truss_spec)
vertical_brace_assign = assign_profile(vertical_brace_ids,profile_ids[PiperackMembers.VerticalBracing])

In [16]:

stub_braces = []
lower_stubs = []
intermediate_transverse_beams_with_stub = []

# Create stub beams between adjacent longitudinal beams
create_stub = lambda beamA,beamB : Beam3D(start=beamA.mid(),end=beamB.mid())

for long_beam_i in range(len(longitudinal_beams)-1):
    long_ii:Beam3D = longitudinal_beams[long_beam_i]
    long_ij:Beam3D = longitudinal_beams[long_beam_i+1]
    if(long_ii.start.eq_x(long_ij.start) and long_ii.end.eq_z(long_ij.end)):
        stubs.append(create_stub(long_ii,long_ij))

if(len(tiers)>0):
    lowest_tier = tiers[0]
    lowest_tier_y = lowest_tier.base.y
    
    if(lowest_tier.intermediate_transverse_beam):
        lowest_long_beam = sorted(longitudinal_beams,key = lambda beam : beam.start.y)[0]
        
        if lowest_long_beam.start.y in long_beams_y:
            lowest_long_beams = long_beams_y[lowest_long_beam.start.y]

            ht_diff = lowest_long_beam.start.y - lowest_tier_y
            
            for long_beam_i in range(0, len(lowest_long_beams), 2):
                first_beam : Beam3D = lowest_long_beams[long_beam_i]
                second_beam :Beam3D = lowest_long_beams[long_beam_i + 1] if long_beam_i + 1 < len(lowest_long_beams) else None
                
                first_beam_mid = first_beam.mid()
                second_beam_mid = second_beam.mid()
                
                lower_stubs.append(Beam3D(start=first_beam_mid.shift_y(lowest_tier_y),end=first_beam_mid.shift_y(lowest_long_beam.start.y)))
                lower_stubs.append(Beam3D(start=second_beam_mid.shift_y(lowest_tier_y),end=second_beam_mid.shift_y(lowest_long_beam.start.y)))
                                
                stub_braces.append(Beam3D(start=first_beam_mid.shift_y(lowest_tier_y),end=first_beam_mid.shift_y(lowest_long_beam.start.y).shift_z(first_beam_mid.z+ht_diff)))
                stub_braces.append(Beam3D(start=second_beam_mid.shift_y(lowest_tier_y),end=second_beam_mid.shift_y(lowest_long_beam.start.y).shift_z(first_beam_mid.z+ht_diff)))

                intermediate_transverse_beams_with_stub.append(Beam3D(start=first_beam_mid,end=second_beam_mid).shift_to_y(lowest_tier_y))

stubs = beams_sorted_yzx(stubs)

# Add stubs and assign properties
stub_ids = add_beams(stubs)
stub_assign = assign_profile(stub_ids,profile_ids[PiperackMembers.Stubs])
stub_release = set_start_end_release(stub_ids)

lower_stub_ids = add_beams(lower_stubs)
lower_stub_assign = assign_profile(lower_stub_ids,profile_ids[PiperackMembers.Stubs])
lower_stub_release = assign_specification(lower_stub_ids,end_release_spec)

stub_brace_ids = add_beams(stub_braces)
stub_assign = assign_profile(stub_brace_ids,profile_ids[PiperackMembers.VerticalBracing])
stub_brace_spec = assign_specification(stub_brace_ids,truss_spec)

intermediate_transverse_beams_with_stub_ids = add_beams(intermediate_transverse_beams_with_stub)
intermediate_transverse_beams_with_stub_assign = assign_profile(intermediate_transverse_beams_with_stub_ids,profile_ids[PiperackMembers.IntermediateTransverseBeams])

In [17]:
# Create intermediate transverse beams from stubs
for stub_i in range(0, len(stubs), 2):
    if stub_i + 1 < len(stubs):  
        stub1, stub2 = stubs[stub_i], stubs[stub_i + 1]
        stub1_mid,stub2_mid = stub1.mid(),stub2.mid()
        
        for tier_y,tier_x in tier_dict.items():
            if( stub1.start.y <= tier_y <= stub1.end.y and tier_x.intermediate_transverse_beam):
                intermediate_transverse_beams.append(Beam3D(start=stub1_mid,end=stub2_mid)\
                                                        .shift_to_y(tier_y))
                                                        # .shift_to_y(closest_to(tier_elevations,stub1_mid.mid(stub2_mid).y)))
    
# Add intermediate transverse beams and assign properties
intermediate_transverse_ids = add_beams(intermediate_transverse_beams)
inter_trans_assign = assign_profile(intermediate_transverse_ids,profile_ids[PiperackMembers.IntermediateTransverseBeams])
inter_trans_release = set_start_end_release(intermediate_transverse_ids)

# Create intermediate longitudinal beams and plan braces from portal beams
portal_beams_y = group_beams_by_y([beam for portal in portals for beam in portal.beams])
for tier_y,tier_x in tier_dict.items():
    if(tier_y in portal_beams_y):
        sorted_beams = beams_sorted_yzx(portal_beams_y[tier_y])
        
        for beam_i in range(len(sorted_beams)-1):
            beam1 : Beam3D = sorted_beams[beam_i]
            beam2 : Beam3D = sorted_beams[beam_i+1]

            if(tier_x.tier_type == TierType.Piping):
                beam1_mid = beam1.mid()
                beam2_mid = beam2.mid()

                if(tier_x.intermediate_transverse_beam):
                    beam_mid = beam1_mid.mid(beam2_mid)
                    intermediate_long_beams.append(Beam3D(start=beam1_mid,end=beam_mid))
                    intermediate_long_beams.append(Beam3D(start=beam_mid,end=beam2_mid))
                else:
                    intermediate_long_beams.append(Beam3D(start=beam1_mid,end=beam2_mid))

                if(braces_placement[beam_i]):
                    mid_pt = beam1.start.mid(beam2.end)
                    for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                        plan_braces.append(Beam3D(start=pt_x,end=mid_pt))

            elif(tier_x.tier_type == TierType.Flare): 
                if(braces_placement[beam_i]):
                    flare_level_x = beam1.start.mid(beam2.end)
                    
                    for line in flare_support_members:
                        if(line.start.eq_y(tier_y)):
                            flare_level_x = beam1.start.mid(beam2.end).shift_x(line.start.x)

                    mid_pt = flare_level_x
                    for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                        plan_braces.append(Beam3D(start=pt_x,end=mid_pt))

            elif(tier_x.tier_type == TierType.ElectricalIntrumentation):
                if(braces_placement[beam_i]):
                    tree_level_x = beam1.start.mid(beam2.end)
                    
                    if(tree_support_members):
                        for line in tree_support_members:
                            if(line.start.eq_y(tier_y)):
                                tree_level_x = beam1.start.mid(beam2.end).shift_x(line.start.x)
                    elif(not tree_support_members and duct_support_members):
                        for line in duct_support_members:
                            if(line.start.eq_y(tier_y)):
                                tree_level_x = beam1.start.mid(beam2.end).shift_x(line.start.x)

                    mid_pt = tree_level_x
                    for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                        plan_braces.append(Beam3D(start=pt_x,end=mid_pt))


# Add intermediate longitudinal beams and assign properties
intermediate_long_ids = add_beams(intermediate_long_beams)
inter_long_assign = assign_profile(intermediate_long_ids,profile_ids[PiperackMembers.IntermediateLongitudinalBeams])
inter_long_release = set_start_end_release(intermediate_long_ids)

# Add plan braces and assign properties
plan_brace_ids = add_beams(plan_braces)
plan_brace_spec = assign_specification(plan_brace_ids,truss_spec)
plan_brace_assign = assign_profile(plan_brace_ids,profile_ids[PiperackMembers.PlanBracing])

# Create comprehensive markdown table for secondary members
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types              | IDs                        | Profile Assignment      | Release Assignment      | Specification           | Function |\n"
markdown_output += "| ---                       | ---                        | ---                     | ---                     | ---                     | ---      |\n"
markdown_output += f"| Stubs                    | {stub_ids}                 | {stub_assign}          | {stub_release}          | -                       | Pipe support connections |\n"
markdown_output += f"| Intermediate Trans Beams | {intermediate_transverse_ids} | {inter_trans_assign} | {inter_trans_release}   | -                       | Secondary transverse support |\n"
markdown_output += f"| Intermediate Long Beams  | {intermediate_long_ids}    | {inter_long_assign}    | {inter_long_release}    | -                       | Secondary longitudinal support |\n"
markdown_output += f"| Plan Braces              | {plan_brace_ids}           | {plan_brace_assign}    | -                       | {plan_brace_spec}       | Horizontal stability bracing |\n"
markdown_output += f"| Vertical Braces          | {vertical_brace_ids}       | {vertical_brace_assign}| -                       | {vertical_brace_spec}   | Vertical stability bracing |\n"

# Add analysis summary
markdown_output += "\n## Secondary Members Analysis\n\n"
markdown_output += f"- **Bracing Pattern**: {brace_pattern}\n"
markdown_output += f"- **Total Stubs**: {len(stub_ids)} (connecting longitudinal beam pairs)\n"
markdown_output += f"- **Total Intermediate Transverse**: {len(intermediate_transverse_ids)} (from stub midpoints)\n"
markdown_output += f"- **Total Intermediate Longitudinal**: {len(intermediate_long_ids)} (from portal beam midpoints)\n"
markdown_output += f"- **Total Plan Braces**: {len(plan_brace_ids)} (horizontal X-bracing)\n"
markdown_output += f"- **Total Vertical Braces**: {len(vertical_brace_ids)} (vertical V-bracing)\n"

# display(Markdown(markdown_output))

In [18]:
# Flare support Members
flare_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.Flare):
        line:Line3D
        for line in flare_support_members:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    flare_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

flare_long_ids = add_beams(flare_members)
flare_long_assign = assign_profile(flare_long_ids,profile_ids[PiperackMembers.FlareSupportMembers])
flare_long_release = set_start_end_release(flare_long_ids)

# Tree Support Members
tree_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.ElectricalIntrumentation):
        line:Line3D
        for line in tree_support_members:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    tree_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

tree_ids = add_beams(tree_members)
tree_assign = assign_profile(tree_ids,profile_ids[PiperackMembers.TreeSupportMembers])
tree_release = set_start_end_release(tree_ids)

# Duct Support Members
duct_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.ElectricalIntrumentation):
        line:Line3D
        for line in duct_support_members:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    duct_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),
                                             end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

duct_ids = add_beams(duct_members)
duct_assign = assign_profile(duct_ids, profile_ids[PiperackMembers.DuctSupportMembers])
duct_release = set_start_end_release(duct_ids)

# WW Support Members
ww_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.ElectricalIntrumentation):
        line:Line3D
        for line in ww_support_members:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    ww_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),
                                           end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

ww_ids = add_beams(ww_members)
ww_assign = assign_profile(ww_ids, profile_ids[PiperackMembers.WWSupportMembers])
ww_release = set_start_end_release(ww_ids)

# Updated Markdown output
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types              | IDs                        | Profile Assignment      | Release Assignment      | Specification           | cláss    |\n"
markdown_output += "| ---                       | ---                        | ---                     | ---                     | ---                     | ---      |\n"
markdown_output += f"| Flare Members            | {flare_long_ids}       | {flare_long_assign}        | {flare_long_release}     | -                      | Flare Support |\n"
markdown_output += f"| Tree Members             | {tree_ids}             | {tree_assign}             | {tree_release}          | -                      | Tree Support |\n"
markdown_output += f"| Duct Members             | {duct_ids}             | {duct_assign}             | {duct_release}          | -                      | Duct Support |\n"
markdown_output += f"| WW Members          | {ww_ids}               | {ww_assign}               | {ww_release}            | -                      | WW Support |\n"

display(Markdown(markdown_output))

## Secondary Members Created

| Member Types              | IDs                        | Profile Assignment      | Release Assignment      | Specification           | cláss    |
| ---                       | ---                        | ---                     | ---                     | ---                     | ---      |
| Flare Members            | [575, 577, 579, 581, 583, 585]       | [True, True, True, True, True, True]        | [(True, True), (True, True), (True, True), (True, True), (True, True), (True, True)]     | -                      | Flare Support |
| Tree Members             | [588, 590, 592, 594, 596, 598]             | [True, True, True, True, True, True]             | [(True, True), (True, True), (True, True), (True, True), (True, True), (True, True)]          | -                      | Tree Support |
| Duct Members             | [601, 603, 605, 607, 609, 611, 614, 616, 618, 620, 622, 624]             | [True, True, True, True, True, True, True, True, True, True, True, True]             | [(True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True)]          | -                      | Duct Support |
| WW Members          | [627, 629, 631, 633, 635, 637, 640, 642, 644, 646, 648, 650, 653, 655, 657, 659, 661, 663, 666, 668, 670, 672, 674, 676]               | [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]               | [(True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True), (True, True)]            | -                      | WW Support |


### Brackets


In [19]:
bracket_beams_z = []
bracket_beams_x = []
bracket_braces = []

for tier_x in tiers:
    if(tier_x.bracket_provision):
        for column_x in column_distances:
            bracket_beams_z.append(Beam3D(start=base_point_of_first_portal.shift_y(tier_x.base.y),end=base_point_of_first_portal.shift_y(tier_x.base.y).shift_z(-bracket_size)).shift_to_x(column_x))
            bracket_braces.append(Beam3D(
                start=base_point_of_first_portal.shift_y(get_closest_lower_value_then_higher_value(long_beams_y,tier_x.base.y)),
                end=base_point_of_first_portal.shift_y(tier_x.base.y).shift_z(-bracket_size)).shift_to_x(column_x))
        
        for i in range(0, len(bracket_beams_z), 2):
            if i + 1 < len(bracket_beams_z):  # Ensure we don't go out of bounds

                beam_mid_z1 = bracket_beams_z[i].start.mid(bracket_beams_z[i + 1].start)
                beam_mid_z2 = bracket_beams_z[i].end.mid(bracket_beams_z[i + 1].end)
                beam_mid_z3 = bracket_beams_z[i].end.mid(beam_mid_z2)
                beam_mid_z4 = bracket_beams_z[i + 1].end.mid(beam_mid_z2)
                
                beam_x = Beam3D(start=bracket_beams_z[i].end, end=bracket_beams_z[i + 1].end)
                bracket_beams_x.append(beam_x)

                tier_x.add_bracket(beam_x)
                
                bracket_braces.append(Beam3D(start=bracket_beams_z[i].start,end=beam_mid_z3))
                bracket_braces.append(Beam3D(start=bracket_beams_z[i+1].start,end=beam_mid_z4))

                bracket_braces.append(Beam3D(start=beam_mid_z1,end=beam_mid_z2))
                bracket_braces.append(Beam3D(start=beam_mid_z1,end=beam_mid_z3))
                bracket_braces.append(Beam3D(start=beam_mid_z1,end=beam_mid_z4))

bracket_beam_z_ids = add_beams(bracket_beams_z)
bracket_beam_x_ids = add_beams(bracket_beams_x)

for count,beam in enumerate(bracket_beams_x):
    beam.id = bracket_beam_x_ids[count]

bracket_beam_ids = []
bracket_beam_ids.extend(bracket_beam_x_ids)
bracket_beam_ids.extend(bracket_beam_z_ids)

bracket_beam_ids.extend(add_beams(bracket_beams_x))
bracket_beam_assign = assign_profile(bracket_beam_ids, profile_ids[PiperackMembers.BracketBeams])

bracket_brace_ids = add_beams(bracket_braces)
bracket_brace_assign = assign_profile(bracket_brace_ids, profile_ids[PiperackMembers.BracketBraces])
bracket_brace_spec = assign_specification(bracket_brace_ids,truss_spec)

## Loads

### Initialization


In [20]:
from base.load.wind_load import *

add_conc_forces_to_members = add_conc_forces_to_members_fn(load)
add_uniform_forces_to_members = add_uniform_forces_to_members_fn(load)

def beam_list_select_and_display(beam_list):
    select_beams(beam_list.keys())
    display(beam_list)

staad_format_list = lambda ids : format_consecutive_numbers(group_consecutive_numbers(ids))

def beam_list_copy_and_display(beam_list):
    if(len(beam_list)> 0 ):
        selected_members = staad_format_list(beam_list)
        pyperclip.copy(f'{selected_members}')
        # copy(f'members={selected_members},')
        display(Markdown(f'copied **{len(beam_list)}** members : {selected_members}'))
    else:
        display(Markdown(f'No member selected'))

### Member Import

In [21]:
nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None,nodes=nodes)
beam_nos:list = list(beam_objects.keys())

tier_elevations = [tier.base.y for tier in tiers]
portal_zs = [portal.base.z for portal in portals]

#### Break Beams at all nodes

In [22]:
break_output = break_beams(geometry,list(nodes.keys()))

markdown_output = '#### Beam Breaks at Nodes\n'
markdown_output += '| Beams          | Ids |\n'
markdown_output += '|----------------|-------|\n'

for k, v in break_output.items():
    markdown_output += f'| {k} | {v} |\n'

display(Markdown(markdown_output))

#### Beam Breaks at Nodes
| Beams          | Ids |
|----------------|-------|
| existing_beams | [(7, 8, 71, 87, 74, 90, 77, 93, 80, 96, 17, 18, 72, 88, 75, 91, 78, 94, 81, 97, 27, 28, 103, 114, 105, 116, 107, 118, 109, 120, 37, 38, 125, 136, 127, 138, 129, 140, 131, 142, 47, 48, 147, 158, 149, 160, 151, 162, 153, 164, 57, 58, 169, 180, 171, 182, 173, 184, 175, 186, 67, 68, 191, 202, 193, 204, 195, 206, 197, 208, 590, 596, 577, 583)] |
| new_beams | [(703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776)] |


### Selfweight (Limited Functionality)

> **Workaround:** Selfweight alloted to all the members. Select **steel members** in the **Selector** code and copy the members and assign using **Assign to edit list**. Same for **concrete members**.

In [23]:
set_load_case_active(STAAD_objects.load,load_case_no=LoadCase.SelfWeight)

add_selfweight_xyz(STAAD_objects.load,factor=-1)    ## For Concrete
add_selfweight_xyz(STAAD_objects.load,factor=-1.1)  ## For Steel + Connections

True

### Portal Beams And Intermediate Beams

#### Segregation

In [24]:
# Levels matching with tier levels and z position with portal positions
portal_beams = {}
intermediate_transverse_beams = {}

portal_beam_dict = {portal.base.z:[] for portal in portals}
portal_tier_beams = {portal:{tier:[] for tier in tier_elevations} for portal in portal_zs}

for beam_no,beam in beam_objects.items():
    
    tier_y = None
    portal_z = None
    chosen_one_1,chosen_one_2 = False,False
    
    beam:Beam3D
    
    for tier_x in tier_elevations:
        if(beam.start.eq_z(beam.end) and beam.start.eq_y(tier_x) and beam.end.eq_y(tier_x)):
            chosen_one_1 = True
            tier_y = tier_x
    
    for portal in portal_zs:
        if(beam.start.eq_z(portal) and beam.end.eq_z(portal)):
            chosen_one_2 = True
            portal_z = portal
    
    if(chosen_one_1 and chosen_one_2):
        portal_beams[beam_no] = beam
        if(tier_y and tier_x in tier_dict):
            tier_dict[tier_y].add_beam(beam)
        if(portal_z and portal_z in portal_beam_dict):
            portal_beam_dict[portal_z].append(beam)
            
        portal_tier_beams[portal_z][tier_y].append(beam_no)
        
    if(chosen_one_1 and not chosen_one_2):
        intermediate_transverse_beams[beam_no] = beam
        if(tier_y in tier_dict):
            tier_dict[tier_y].add_int_beam(beam)

# beam_list_select_and_display(portal_beams)
# time.sleep(2)
# beam_list_select_and_display(intermediate_transverse_beams)

#### Load Assignment

##### Portal Beams

In [25]:
for tier_x in tiers:
    for load_x in tier_x.loads:
        set_load_case_active(STAAD_objects.load,load_x.load_case)
        
        if(isinstance(load_x,ConcentratedLoad)):
            add_conc_forces_to_members(tier_x.beams,load_x)

max_portal_to_portal = max([portals[portal_i].base.distance_to(portals[portal_i+1].base) for portal_i in range(len(portals)-1)])

for portal_i in range(len(portals)):
    current_portal = portals[portal_i]
    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
    next_portal = portals[portal_i + 1] if portal_i < len(portals) - 1 else None
    
    dist_1 = previous_portal.base.distance_to(current_portal.base) if previous_portal else max_portal_to_portal
    dist_2 = next_portal.base.distance_to(current_portal.base) if next_portal else max_portal_to_portal
    
    for tier_x in tiers:
        for load_x in tier_x.loads:
            set_load_case_active(STAAD_objects.load,load_x.load_case)
            
            if(isinstance(load_x,UniformLoad)):
                add_uniform_forces_to_members(portal_tier_beams[current_portal.base.z][tier_x.base.y],load_x.factor_force_value(avg([dist_1,dist_2])))

##### Intermediate Transverse Beams

In [26]:
for tier_x in tiers:
    for load_x in tier_x.clt_loads:
        set_load_case_active(STAAD_objects.load,load_x.load_case)
        
        if(isinstance(load_x,ConcentratedLoad)):
            add_conc_forces_to_members(tier_x.int_beams,load_x)

for tier_x in tiers:
    tier_sorted_z = sorted(tier_x.int_beams,key=lambda beam : beam.start.z)
    for beam in tier_sorted_z:

        closest_portals = sorted(portals,key=lambda portal : portal.base.distance_to(beam.mid()))
        distance = closest_portals[0].base.distance_to(closest_portals[1].base)
        # select_beam(geometry,beam.id)

        for load_x in tier_x.clt_loads:
            set_load_case_active(STAAD_objects.load,load_x.load_case)

            if(isinstance(load_x,UniformLoad)):
                add_uniform_forces_to_members([beam],load_x.factor_force_value(distance))


### Brackets

#### Identification

In [27]:
brackets = [beam for beam in beam_objects.values() if beam.start.eq_z(beam.end) and beam.start.z < base_point_of_first_portal.z]
# select_beams([beam.id for beam in brackets])

brackets_y = sorted(set(beam.start.y for beam in brackets))
brackets_y_dict = {y: [] for y in brackets_y}

for bracket in brackets:
    brackets_y_dict[bracket.start.y].append(bracket)

for tier_x in tiers:
    if(tier_x.base.y in brackets_y_dict):
        for bracket in brackets_y_dict[tier_x.base.y]:
            tier_x.add_bracket(bracket)

#### Assignment

In [28]:
for tier_x in tiers:
    for load_x in tier_x.loads:
        set_load_case_active(STAAD_objects.load,load_x.load_case)
        
        if(isinstance(load_x,UniformLoad)):
            add_uniform_forces_to_members(tier_x.brackets,load_x.factor_force_value(max_expansion_bay_length/2))

### Columns

#### Column Identification

In [29]:
portal_z_set = set(portal_zs)

columns_nos = [beam_no for beam_no,beam in beam_objects.items() if beam.start.x == beam.end.x and beam.start.z == beam.end.z and abs(beam.start.y - beam.end.y) > 0.1]
portal_columns_nos = [beam_no for beam_no,beam in beam_objects.items() if beam.start.x == beam.end.x and beam.start.z == beam.end.z and  beam.start.z in portal_z_set and abs(beam.start.y - beam.end.y) > 0.1]
stubs_column_nos = [beam_no for beam_no in columns_nos if beam_no not in portal_columns_nos]

x_coordinates = sorted(list(set([beam_objects[col_i].start.x for col_i in portal_columns_nos])))
z_coordinates = sorted(list(set([beam_objects[col_i].start.z for col_i in portal_columns_nos])))
y_coordinates = sorted(list(set([beam_objects[col_i].start.y for col_i in portal_columns_nos])))

columns_x = {x:[] for x in x_coordinates}
columns_y = {y:[] for y in y_coordinates}
columns_z = {z:[] for z in z_coordinates}

for col_i in portal_columns_nos:
    beam = beam_objects[col_i]
    columns_x[beam.start.x].append(beam)

for col_i in portal_columns_nos:
    beam = beam_objects[col_i]
    columns_y[beam.start.y].append(beam)

for col_i in portal_columns_nos:
    beam = beam_objects[col_i]
    columns_z[beam.start.z].append(beam)

#### Wind Load Definition

In [30]:
ngl = -0.5

add_ngl = add_elevation_to_wind_load_list_fn(ngl)

transverse_wind_loads = [
    WindLoad(start_ele=0.0, end_ele=10.0, windward=0.5408, leeward=0.4501),
    WindLoad(start_ele=10.0, end_ele=15.0, windward=0.5962, leeward=0.4962),
    WindLoad(start_ele=15.0, end_ele=20.0, windward=0.6191, leeward=0.5153),
    WindLoad(start_ele=20.0, end_ele=25.0, windward=0.6484, leeward=0.5396),
    WindLoad(start_ele=25.0, end_ele=30.0, windward=0.6783, leeward=0.5645)
]

longitudinal_wind_loads = [
    WindLoad(start_ele=0.0, end_ele=10.0, windward=0.2704, leeward=0.2279),
    WindLoad(start_ele=10.0, end_ele=15.0, windward=0.2981, leeward=0.2513),
    WindLoad(start_ele=15.0, end_ele=20.0, windward=0.3096, leeward=0.2610),
    WindLoad(start_ele=20.0, end_ele=25.0, windward=0.3242, leeward=0.2733),
    WindLoad(start_ele=25.0, end_ele=30.0, windward=0.3392, leeward=0.2859)
]

add_ngl(transverse_wind_loads)
add_ngl(longitudinal_wind_loads)

display(Markdown(wind_load_definition_markdown(transverse_wind_loads)))
display(Markdown(wind_load_definition_markdown(longitudinal_wind_loads)))

| Start Elevation | End Elevation | Windward Load | Leeward Load |
|-----------------|---------------|---------------|--------------|
| -0.5 | 9.5 | 0.5408 | 0.4501 |
| 9.5 | 14.5 | 0.5962 | 0.4962 |
| 14.5 | 19.5 | 0.6191 | 0.5153 |
| 19.5 | 24.5 | 0.6484 | 0.5396 |
| 24.5 | 29.5 | 0.6783 | 0.5645 |


| Start Elevation | End Elevation | Windward Load | Leeward Load |
|-----------------|---------------|---------------|--------------|
| -0.5 | 9.5 | 0.2704 | 0.2279 |
| 9.5 | 14.5 | 0.2981 | 0.2513 |
| 14.5 | 19.5 | 0.3096 | 0.2610 |
| 19.5 | 24.5 | 0.3242 | 0.2733 |
| 24.5 | 29.5 | 0.3392 | 0.2859 |


#### Wind Load Assignment

##### Transverse Direction

In [31]:
generated_loads = []
set_load_case_active(load,LoadCase.WindColumn_GX)
pedestal_offset_start_y = base_point_of_first_portal.y

for z in x_coordinates:
    for col in columns_x[z]:
        for wind_l in transverse_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)

                force = wind_l.windward if z==x_coordinates[0] else wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX))

set_load_case_active(load,LoadCase.WindColumn_GX_Opposite)
for z in x_coordinates:
    for col in columns_x[z]:
        for wind_l in transverse_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                
                force = -wind_l.windward if z==x_coordinates[-1] else -wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX_Opposite)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX_Opposite))

# display(Markdown(wind_load_assignment_markdown('Wind GX Forces on Columns',generated_loads)))

##### Longitudinal Direction

In [32]:
generated_loads = []
set_load_case_active(load,LoadCase.WindColumn_GZ)

for z in z_coordinates:
    for col in columns_z[z]:
        for wind_l in longitudinal_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)

                force = wind_l.windward if z==z_coordinates[0] else wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ))

set_load_case_active(load,LoadCase.WindColumn_GZ_Opposite)

for z in z_coordinates:
    for col in columns_z[z]:
        for wind_l in longitudinal_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                
                force = -wind_l.windward if z==z_coordinates[-1] else -wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ_Opposite)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ_Opposite))

# display(Markdown(wind_load_assignment_markdown('Wind GZ Forces on Columns',generated_loads)))

##### Tier-wise

In [33]:
for tier_x in tiers:
    beams_by_z = group_beams_by_z(tier_x.beams)

    for z,beams in beams_by_z.items():
        start_beam = min(beams,key=lambda beam: min(beam.start.x, beam.end.x))
        end_beam = max(beams,key=lambda beam: max(beam.start.x, beam.end.x))

        min_x = min(start_beam.start.x, start_beam.end.x)
        max_x = max(end_beam.start.x, end_beam.end.x)
        
        application_node_pt_1 = tier_x.base.shift_x(min_x).shift_z(z)
        tier_x_loads = [wl for wl in tier_x.wind_loads if wl.load_case == LoadCase.WindTier_GX]
        for load_x in tier_x_loads:
            set_load_case_active(load,load_x.load_case)
            add_point_conc_force(load=load,geometry=geometry,point=application_node_pt_1,load_object=load_x)  

        application_node_pt_2 = tier_x.base.shift_x(max_x).shift_z(z)
        tier_x0_loads = [wl for wl in tier_x.wind_loads if wl.load_case == LoadCase.WindTier_GX_Opposite]
        for load_x in tier_x0_loads:
            set_load_case_active(load,load_x.load_case)
            add_point_conc_force(load=load,geometry=geometry,point=application_node_pt_2,load_object=load_x)  

### Post Build Script

In [34]:
openSTAAD.UpdateStructure()
time.sleep(10)

## Member Selector

In [35]:
nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None,nodes=nodes)
beam_nos:list = list(beam_objects.keys())

tier_elevations = [tier.base.y for tier in tiers]
portal_zs = [portal.base.z for portal in portals]

#### Steel Members

In [36]:
steel_members = [beam for beam in beam_objects.values() if beam.start.y >= base_point_of_first_portal.y]
beam_list_copy_and_display([beam.id for beam in steel_members])

copied **762** members : 1 To 8 11 To 18 21 To 28 31 To 38 41 To 48 51 To 58 61 To 68 71 To 776

#### Concrete Members

In [37]:
concrete_members = [beam for beam in beam_objects.values() if beam.start.y < base_point_of_first_portal.y]
beam_list_copy_and_display([beam.id for beam in concrete_members])

copied **14** members : 9 To 10 19 To 20 29 To 30 39 To 40 49 To 50 59 To 60 69 To 70

### Material Assignment 

In [38]:
%%capture
assign_material_steel([beam.id for beam in steel_members])
assign_material_concrete([beam.id for beam in concrete_members])

### Show Selected Members

In [39]:
selected_members = get_selected_beam_nos(geometry)
beam_list_copy_and_display(selected_members)

No member selected

### Tier Details (Full)

In [40]:
# for tier in tiers:
#     display(Markdown(tier.to_markdown()))

### Column Details

In [41]:
markdown_output = '### Column Table\n'
markdown_output += '| Elevation | Columns |\n'
markdown_output += '|-----------|---------|\n'

for z, column in columns_y.items():
    beam_ids = ', '.join([str(beam.id) for beam in column])
    markdown_output += f'| {z} | {beam_ids} |\n'

display(Markdown(markdown_output))

### Column Table
| Elevation | Columns |
|-----------|---------|
| -2.0 | 9, 10, 19, 20, 29, 30, 39, 40, 49, 50, 59, 60, 69, 70 |
| 0.0 | 7, 8, 17, 18, 27, 28, 37, 38, 47, 48, 57, 58, 67, 68 |
| 4.5 | 71, 72, 87, 88, 103, 114, 125, 136, 147, 158, 169, 180, 191, 202 |
| 7.75 | 74, 75, 90, 91, 105, 116, 127, 138, 149, 160, 171, 182, 193, 204 |
| 10.75 | 77, 78, 93, 94, 107, 118, 129, 140, 151, 162, 173, 184, 195, 206 |
| 13.25 | 80, 81, 96, 97, 109, 120, 131, 142, 153, 164, 175, 186, 197, 208 |
| 16.0 | 83, 84, 99, 100, 111, 122, 133, 144, 155, 166, 177, 188, 199, 210 |


### Tier Details (Concise)

In [42]:
markdown_output = '### Tier Beams Table\n'
markdown_output += '| Tier Elevation | Count | Beams |\n'
markdown_output += '|-----------|---------| -------|\n'

for tier_x in tiers:
    markdown_output += tier_x.beams_to_markdown()

display(Markdown(markdown_output))

markdown_output = '### Tier Intermediate Beams Table\n'
markdown_output += '| Tier Elevation | Count | Int. Beams |\n'
markdown_output += '|-----------|---------| -------|\n'

for tier_x in tiers:
    markdown_output += tier_x.int_beams_to_markdown()

display(Markdown(markdown_output))

### Tier Beams Table
| Tier Elevation | Count | Beams |
|-----------|---------| -------|
| 3 | 14 | 1, 11, 21, 31, 41, 51, 61, 445, 446, 448, 450, 452, 454, 456 |
| 6 | 14 | 2, 12, 22, 32, 42, 52, 62, 458, 459, 461, 463, 465, 467, 469 |
| 9.5 | 14 | 3, 13, 23, 33, 43, 53, 63, 471, 474, 478, 482, 486, 490, 494 |
| 12 | 14 | 4, 14, 24, 34, 44, 54, 64, 496, 499, 503, 507, 511, 515, 519 |
| 14.5 | 56 | 5, 15, 25, 35, 45, 55, 65, 586, 587, 589, 591, 593, 595, 597, 599, 600, 602, 604, 606, 608, 610, 612, 613, 615, 617, 619, 621, 623, 625, 626, 628, 630, 632, 634, 636, 638, 639, 641, 643, 645, 647, 649, 651, 652, 654, 656, 658, 660, 662, 664, 665, 667, 669, 671, 673, 675 |
| 17.5 | 14 | 6, 16, 26, 36, 46, 56, 66, 573, 574, 576, 578, 580, 582, 584 |


### Tier Intermediate Beams Table
| Tier Elevation | Count | Int. Beams |
|-----------|---------| -------|
| 3 | 0 | None |
| 6 | 0 | None |
| 9.5 | 16 | 411, 414, 417, 420, 423, 426, 472, 476, 480, 484, 488, 492, 681, 685, 687, 689 |
| 12 | 16 | 429, 432, 435, 438, 441, 444, 497, 501, 505, 509, 513, 517, 682, 695, 697, 699 |
| 14.5 | 0 | None |
| 17.5 | 0 | None |


### Vertical Braces

In [43]:
vertical_braces = [beam for beam in beam_objects.values() if (not beam.start.eq_y(beam.end) and beam.start.eq_x(beam.end) and not beam.start.eq_z(beam.end))]

columns_z = defaultdict(list)
for beam in vertical_braces:
    columns_z[beam.start.y].append(beam)

markdown_output = '### Vertical Braces\n'
markdown_output += '| Elevation | Braces |\n'
markdown_output += '|-----------|---------|\n'
for z in sorted(columns_z.keys()):  # Sort elevations in ascending order
    beam_ids = ', '.join([str(beam.id) for beam in columns_z[z]])
    markdown_output += f'| {z} | {beam_ids} |\n'

display(Markdown(markdown_output))


### Vertical Braces
| Elevation | Braces |
|-----------|---------|
| 0.0 | 214, 215, 219, 220, 224, 225, 229, 230 |
| 7.75 | 216, 217, 221, 222, 226, 227, 231, 232, 234, 235, 239, 240, 244, 245, 249, 250, 683, 684 |
| 10.75 | 693, 694 |
| 13.25 | 236, 237, 241, 242, 246, 247, 251, 252, 254, 255, 263, 264, 272, 273, 281, 282 |
| 16.0 | 260, 261, 269, 270, 278, 279, 287, 288 |
| 17.5 | 257, 259, 266, 268, 275, 277, 284, 286 |


### Plan Braces

In [44]:
plan_braces = [beam for beam in beam_objects.values() if (beam.start.eq_y(beam.end) and not beam.start.eq_x(beam.end) and not beam.start.eq_z(beam.end))]

from collections import defaultdict
columns_z = defaultdict(list)
for beam in vertical_braces:
    columns_z[beam.start.y].append(beam)

markdown_output = '### Plan Braces\n'
markdown_output += '| Elevation | Braces |\n'
markdown_output += '|-----------|---------|\n'
for z in sorted(columns_z.keys()):  # Sort elevations in ascending order
    beam_ids = ', '.join([str(beam.id) for beam in columns_z[z]])
    markdown_output += f'| {z} | {beam_ids} |\n'

display(Markdown(markdown_output))

### Plan Braces
| Elevation | Braces |
|-----------|---------|
| 0.0 | 214, 215, 219, 220, 224, 225, 229, 230 |
| 7.75 | 216, 217, 221, 222, 226, 227, 231, 232, 234, 235, 239, 240, 244, 245, 249, 250, 683, 684 |
| 10.75 | 693, 694 |
| 13.25 | 236, 237, 241, 242, 246, 247, 251, 252, 254, 255, 263, 264, 272, 273, 281, 282 |
| 16.0 | 260, 261, 269, 270, 278, 279, 287, 288 |
| 17.5 | 257, 259, 266, 268, 275, 277, 284, 286 |


### Flare Support Beams

In [45]:
member_ids = {tier.base.y : [] for tier in tiers if tier.tier_type == TierType.Flare}
for tier_x in tiers:
    tier_long_beams = [beam for beam in beam_objects.values() if (beam.start.eq_y(tier_x.base) and beam.start.eq_x(beam.end))]
    if tier_x.tier_type == TierType.Flare:
        for flare in flares:
            if flare.support_member and flare.lines:
                for line in flare.lines:
                    if(line.start.eq_y(tier_x.base)):
                        selected_long_beams = [beam for beam in tier_long_beams if beam.start.eq_x(line.start)]
                        member_ids[tier_x.base.y].extend(selected_long_beams)
                        flare.add_members(selected_long_beams)

markdown_output = '### Flare Beams \n'
markdown_output += '| Elevation | Flare Long Beams |\n'
markdown_output += '|-----------|---------|\n'
for z in sorted(member_ids.keys()):  # Sort elevations in ascending order
    beam_ids = ', '.join([str(beam.id) for beam in member_ids[z]])
    markdown_output += f'| {z} | {beam_ids} |\n'

display(Markdown(markdown_output))
beam_list_copy_and_display([beam.id for flare in flares for beam in flare.members])

### Flare Beams 
| Elevation | Flare Long Beams |
|-----------|---------|
| 17.5 | 575, 577, 579, 581, 583, 585, 775, 776 |


copied **8** members : 575 577 579 581 583 585 775 To 776

#### Flare Loads

In [46]:
flares_w_support = [flare for flare in flares if flare.support_member]

for flare in flares_w_support:
    flare.loads.append(uniform_operating_load.set_force_value(flare.design_load*-1))
    flare.loads.append(uniform_empty_load.set_force_value(flare.design_load*-0.4))
    flare.loads.append(uniform_tg_gx.set_force_value(flare.design_load*0.3))
    flare.loads.append(uniform_tl_gx.set_force_value(flare.design_load*0.3))
    flare.loads.append(uniform_tg_gz.set_force_value(flare.design_load*0.3))
    flare.loads.append(uniform_tl_gz.set_force_value(flare.design_load*0.3))

    for f_load in flare.loads:
        set_load_case_active(load,f_load.load_case)
        add_uniform_forces_to_members(flare.members,f_load)

In [47]:

flares_wo_support = [flare for flare in flares if (not flare.support_member)]

for flare in flares_wo_support:
    for portal_i in range(len(portals)):
        current_portal = portals[portal_i]
        previous_portal = portals[portal_i - 1] if portal_i > 0 else None
        next_portal = portals[portal_i + 1] if portal_i < len(portals) - 1 else None
        
        dist_1 = previous_portal.base.distance_to(current_portal.base) if previous_portal else max_portal_to_portal
        dist_2 = next_portal.base.distance_to(current_portal.base) if next_portal else max_portal_to_portal
        load_span = avg([dist_1,dist_2])

        load_point = current_portal.base.shift_x(flare.lines[0].start.x).shift_y(flare.lines[0].start.y)

        flare_loads = [
            conc_operating_load.set_global_apply_point(load_point).set_force_value(flare.design_load*load_span*-1),
            conc_empty_load.set_global_apply_point(load_point).set_force_value(flare.design_load*load_span*-0.4),
            conc_tg_gx.set_global_apply_point(load_point).set_force_value(flare.design_load*load_span*0.3),
            conc_tg_gz.set_global_apply_point(load_point).set_force_value(flare.design_load*load_span*0.3),
            conc_tl_gx.set_global_apply_point(load_point).set_force_value(flare.design_load*load_span*0.3),
            conc_tl_gz.set_global_apply_point(load_point).set_force_value(flare.design_load*load_span*0.3),
        ]
        
        current_portal_beam_nos = portal_tier_beams[current_portal.base.z][tier_x.base.y]
        for beam_no in current_portal_beam_nos:
            beam_line = beam_objects[beam_no].get_line()
            for flare_load_x in flare_loads:
                if beam_line and flare_load_x.set_d1_from_global_point(beam_line):
                    set_load_case_active(load=load,load_case_no=flare_load_x.load_case)
                    add_member_conc_force(load=load,BeamNo=beam_no,load_object=flare_load_x)

### WW Beams

In [48]:
member_ids = {tier.base.y : [] for tier in tiers if tier.tier_type == TierType.ElectricalIntrumentation}
for tier_x in tiers:
    tier_long_beams = [beam for beam in beam_objects.values() if (beam.start.eq_y(tier_x.base) and beam.start.eq_x(beam.end))]
    if tier_x.tier_type == TierType.ElectricalIntrumentation:
        for ww in walkways:
            for line in ww.get_member_lines():
                if(line.start.eq_y(tier_x.base)):
                    selected_long_beams = [beam for beam in tier_long_beams if beam.start.eq_x(line.start)]
                    member_ids[tier_x.base.y].extend(selected_long_beams)
                    ww.add_members(selected_long_beams)

markdown_output = '### WW Beams \n'
markdown_output += '| Elevation | WW Long Beams |\n'
markdown_output += '|-----------|---------|\n'
for z in sorted(member_ids.keys()):  # Sort elevations in ascending order
    beam_ids = ', '.join([str(beam.id) for beam in member_ids[z]])
    markdown_output += f'| {z} | {beam_ids} |\n'

display(Markdown(markdown_output))
beam_list_copy_and_display([beam.id for obj in walkways for beam in obj.members])

### WW Beams 
| Elevation | WW Long Beams |
|-----------|---------|
| 14.5 | 627, 629, 631, 633, 635, 637, 640, 642, 644, 646, 648, 650, 653, 655, 657, 659, 661, 663, 666, 668, 670, 672, 674, 676 |


copied **24** members : 627 629 631 633 635 637 640 642 644 646 648 650 653 655 657 659 661 663 666 668 670 672 674 676

#### WW Loads

In [49]:
for ww in walkways:
    ww_load = ww.get_uniform_load()
    set_load_case_active(load,ww_load.load_case)
    add_uniform_forces_to_members(ww.members,ww_load)

### Duct Support Beams

In [50]:
member_ids = {tier.base.y : [] for tier in tiers if tier.tier_type == TierType.ElectricalIntrumentation}
for tier_x in tiers:
    tier_long_beams = [beam for beam in beam_objects.values() if (beam.start.eq_y(tier_x.base) and beam.start.eq_x(beam.end))]
    if tier_x.tier_type == TierType.ElectricalIntrumentation:
        for duct in ducts:
            for line in duct.get_member_lines():
                if(line.start.eq_y(tier_x.base)):
                    selected_long_beams = [beam for beam in tier_long_beams if beam.start.eq_x(line.start)]
                    member_ids[tier_x.base.y].extend(selected_long_beams)
                    duct.add_members(selected_long_beams)

markdown_output = '### Duct Beams \n'
markdown_output += '| Elevation | Duct Long Beams |\n'
markdown_output += '|-----------|---------|\n'
for z in sorted(member_ids.keys()):  # Sort elevations in ascending order
    beam_ids = ', '.join([str(beam.id) for beam in member_ids[z]])
    markdown_output += f'| {z} | {beam_ids} |\n'

display(Markdown(markdown_output))
beam_list_copy_and_display([beam.id for obj in ducts for beam in obj.members])

### Duct Beams 
| Elevation | Duct Long Beams |
|-----------|---------|
| 14.5 | 601, 603, 605, 607, 609, 611, 614, 616, 618, 620, 622, 624 |


copied **12** members : 601 603 605 607 609 611 614 616 618 620 622 624

#### Duct Loads

In [51]:
for duct in ducts:
    duct_load = duct.get_uniform_load()
    set_load_case_active(load,duct_load.load_case)
    add_uniform_forces_to_members(duct.members,duct_load)

### Tree Support Beams

In [52]:
member_ids = {tier.base.y : [] for tier in tiers if tier.tier_type == TierType.ElectricalIntrumentation}
for tier_x in tiers:
    tier_long_beams = [beam for beam in beam_objects.values() if (beam.start.eq_y(tier_x.base) and beam.start.eq_x(beam.end))]
    if tier_x.tier_type == TierType.ElectricalIntrumentation:
        for tree_x in electric_trees:
            if(tree_x.line.start.eq_y(tier_x.base)):
                selected_long_beams = [beam for beam in tier_long_beams if beam.start.eq_x(tree_x.line.start)]
                member_ids[tier_x.base.y].extend(selected_long_beams)
                tree_x.add_members(selected_long_beams)

markdown_output = '### Tree Beams \n'
markdown_output += '| Elevation | Long Beams |\n'
markdown_output += '|-----------|---------|\n'
for z in sorted(member_ids.keys()):  # Sort elevations in ascending order
    beam_ids = ', '.join([str(beam.id) for beam in member_ids[z]])
    markdown_output += f'| {z} | {beam_ids} |\n'

display(Markdown(markdown_output))
beam_list_copy_and_display([beam.id for obj in electric_trees for beam in obj.members])

### Tree Beams 
| Elevation | Long Beams |
|-----------|---------|
| 14.5 | 588, 590, 592, 594, 596, 598, 773, 774 |


copied **8** members : 588 590 592 594 596 598 773 To 774

#### Tree Loads

In [53]:
for tree_x in electric_trees:
    tree_base_load = tree_x.get_tree_load()
    set_load_case_active(load,tree_base_load.load_case)
    tree_loads_bw_portals = []

    sorted_tree_supports = sorted(tree_x.members,key=lambda beam:beam.start.z)
    
    support_on_portal_members = [beam for beam in sorted_tree_supports if beam.start.z in portal_z_set]
    support_on_portal_members_set = set(support_on_portal_members)
    
    # loads b/w portals
    for first, second in zip(portal_zs, portal_zs[1:]):
        # Access first and second items here
        distance = abs(first-second)
        no_trees = math.ceil(distance / tree_x.tree_to_tree_distance)
        tree_to_tree_distance_alloted = round(distance/no_trees,3)
        for i in range(1,no_trees):
            tree_z = first + tree_to_tree_distance_alloted*i
            # display(str(tree_base_load.set_global_apply_point(tree_x.line.start.shift_z(tree_z))))
            tree_loads_bw_portals.append(tree_base_load.set_global_apply_point(tree_x.line.start.shift_z(tree_z)))
    
    #loads on portal
    for member in sorted_tree_supports:
        if member in support_on_portal_members_set:
            add_member_conc_force(load=load,BeamNo=member.id,load_object=tree_base_load)

        for tree_load_x in tree_loads_bw_portals:
            if(tree_load_x.set_d1_from_global_point(member.get_line())):
                add_member_conc_force(load=load,BeamNo=member.id,load_object=tree_load_x)
                # tree_loads_bw_portals.remove(tree_load_x)
                
    # Load on last member
    add_member_conc_force(load=load,BeamNo=support_on_portal_members[-1].id,load_object=tree_base_load.set_d1(support_on_portal_members[-1].length()))

## Parameter Selector

In [54]:
secondary_members = []
secondary_members.extend(stubs_column_nos)
secondary_members.extend([beam.id for tier in tiers for beam in tier.int_beams])

primary_members = list(set(beam_nos) - set(secondary_members))

### Parameter 1 & 3

In [ ]:
# select_beams(primary_members)
beam_list_copy_and_display(primary_members)

copied **660** members : 1 To 290 292 To 293 295 297 299 To 300 302 To 303 305 To 306 308 To 309 311 313 315 To 316 318 To 319 321 323 327 329 331 333 337 339 341 343 345 347 349 351 353 355 357 359 361 363 365 367 371 373 375 377 381 383 385 387 389 391 393 395 397 399 401 403 405 407 445 To 471 473 To 475 477 To 479 481 To 483 485 To 487 489 To 491 493 To 496 498 To 500 502 To 504 506 To 508 510 To 512 514 To 516 518 To 680 683 To 684 686 688 690 To 694 696 698 700 To 776

### Parameter 2 & 4

In [ ]:
# select_beams(secondary_members)
beam_list_copy_and_display(secondary_members)

copied **116** members : 291 294 296 298 301 304 307 310 312 314 317 320 322 324 To 326 328 330 332 334 To 336 338 340 342 344 346 348 350 352 354 356 358 360 362 364 366 368 To 370 372 374 376 378 To 380 382 384 386 388 390 392 394 396 398 400 402 404 406 408 To 444 472 476 480 484 488 492 497 501 505 509 513 517 681 To 682 685 687 689 695 697 699